In [30]:
import torch
from parser_model import ParserModel
from utils.parser_utils import minibatches, load_and_preprocess_data, read_conll
import numpy as np
import pandas as pd

In [2]:
weights_path = 'results/20240516_155227/model.weights'

In [3]:
parser, embeddings, train_data, dev_data, test_data = load_and_preprocess_data(False)

Loading data...
took 1.37 seconds
Building parser...
took 0.89 seconds
Loading pretrained embeddings...
took 2.61 seconds
Vectorizing data...
took 1.19 seconds
Preprocessing training data...
took 27.53 seconds


In [6]:
parser.model = ParserModel(embeddings)
parser.model.load_state_dict(torch.load(weights_path))

<All keys matched successfully>

In [79]:
_, predictions = parser.parse(dev_data)


unvectorized_dataset = read_conll('data/dev.conll')
flat_preds = []
heads = [np.array(s['head']) for s in unvectorized_dataset]
for ds_index, head in enumerate(heads):
    predicted_deps = predictions[ds_index]
    flat_preds.append([x[0] for x in sorted(predicted_deps, key=lambda x:x[1])])

flat_predictions = [p for pred in flat_preds for p in pred]

df = pd.read_csv('data/dev.pred.conll', header=None,  sep='\t')

df[6] = flat_predictions
df.to_csv('data/dev.pred.conll', index=False, sep='\t')

with open('data/dev.pred.conll', "r") as f:
    string = f.read()

string = string.replace('\n1\t', '\n\n1\t')

with open('data/dev.pred.conll', "w") as f:
    f.write(string)


### DONT FORGET TO REMOVE HEADER

1445850it [00:00, 55610586.32it/s]      


In [ ]:
_, predictions = parser.parse(test_data)


unvectorized_dataset = read_conll('data/test.conll')
flat_preds = []
heads = [np.array(s['head']) for s in unvectorized_dataset]
for ds_index, head in enumerate(heads):
    predicted_deps = predictions[ds_index]
    flat_preds.append([x[0] for x in sorted(predicted_deps, key=lambda x:x[1])])

flat_predictions = [p for pred in flat_preds for p in pred]

df = pd.read_csv('data/test.pred.conll', header=None,  sep='\t')

df[6] = flat_predictions
df.to_csv('data/test.pred.conll', index=False, sep='\t')

with open('data/test.pred.conll', "r") as f:
    string = f.read()

string = string.replace('\n1\t', '\n\n1\t')

with open('data/test.pred.conll', "w") as f:
    f.write(string)


### DONT FORGET TO REMOVE HEADER